In [1]:
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
import tensorflow
import ee
import json
from collections import OrderedDict
import time
import math
import csv
import os

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWiACvC5E2c9evdwwh309oKFPcSeozZLhrtp0XXOkMxXznaO9VfSLN4



Successfully saved authorization token.


In [3]:
# Set the test image using an uploaded asset for now
inputImageCollection = ee.ImageCollection('users/kateharvey/test_collection')
# inputImage = ee.Image(inputImageCollection.toList(inputImageCollection.size()).get(0))
# inputImage = ee.Image('users/kateharvey/test_collection/20200801T155911_20200801T160644_T18TVQ_LAI')

# TESTING with 10m
testImage = ee.Image('users/kateharvey/TestImage')
inputImage = ee.Image(testImage.select(1,2,3,7,23,27,28,29,30,31,32,33))

# Change variable name here to match band name pattern (one of: Albedo, fAPAR, fCOVER, LAI, CCC, CWC, DASF)
outputName = 'LAI'
defaultBand = ee.String('estimate'+outputName)

# List the bands that we expect in the image (ensure number of bands in the list below matches the number bands in the input image)
# This is the order of bands produced by running SL2P10 and exporting the resulting ImageCollection
# inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'date', 'QC', 'estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1'])

inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'date', 'QC', 'estimateLAI', 'partition', 'networkID', 'errorLAI', 'partition_1', 'networkID_1'])

In [4]:
# Below we define a list of strings representing the expressions for each vegetation index as a function of the bands in the input image
# More vegetation indices can be defined, but the list CANNOT contain any two vegetation indices which are a linear combination of each
# other or LARs will fail to select the requested number of variables

# The formatting of the expression must be
# "<name of VI> = <expression with band names from inputImage_bands used as variables in the form b('<band name>')"

# Only include VIs that use 10 m bands (B2, B3, B4, B8)
input_VI_definition = ee.List(["RAW_B2  = b('B2')",
                               "RAW_B3  = b('B3')",
                               "RAW_B4  = b('B4')",
                               "RAW_B8  = b('B8')",
                               "GI      = b('B3')/b('B4')",
                             # "RVI3    = b('B4')/b('B6')",
                             # "SR3     = b('B5')/b('B4')",
                             # "GM1     = b('B6')/b('B3')",
                             # "GM2     = b('B6')/b('B5')",
                             # "SR2     = b('B7')/b('B3')",
                             # "PSSR    = b('B7')/b('B4')",
                               "SGI     = b('B8')/b('B4')",
                             # "MSI     = b('B11')/b('B7')",
                             # "II      = b('B11')/b('B12')",
                               "GVI     = (b('B8')/b('B3'))-1",
                             # "PSRI    = (b('B4')-b('B3'))/b('B6')",
                               "NDVI3   = ((b('B8')-b('B4'))/(b('B8')))+b('B4')",
                             # "SR5     = 1/b('B5')",
                             # "SR6     = b('B4')/(b('B3')*b('B5'))",
                             # "SR7     = b('B8')/(b('B3')*b('B5'))",
                             # "IPVI    = b('B7')/(b('B7')+b('B4'))",
                             # "ARI     = (1/b('B3'))-(1/b('B5'))",
                             # "ARI2    = b('B7')*((1/b('B3'))-(1/b('B5')))",
                               "NDVI    = (b('B8')-b('B4'))/(b('B8')+b('B4'))",
                               "GNDVI   = (b('B8')-b('B3'))/(b('B8')+b('B3'))",
                             # "NDWI    = (b('B8')-b('B11'))/(b('B8')+b('B11'))",
                             # "NDREVI  = (b('B8')-b('B5'))/(b('B8')+b('B5'))",
                               "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",
                             # "NDI1    = (b('B7')-b('B5'))/(b('B7')-b('B4'))",
                             # "NDI2    = (b('B8')-b('B5'))/(b('B8')-b('B4'))",
                             # "RENDVI  = (b('B6')-b('B5'))/(b('B6')+b('B5'))",
                             # "OSAVI   = (1.16*(b('B7')-b('B4')))/(b('B7')+b('B4')+0.61)",
                             # "NMDI    = (b('B8')-(b('B11')-b('B12')))/(b('B8')+(b('B11')-b('B12')))",
                             # "HI      = ((b('B3')-b('B5'))/(b('B3')+b('B5')))-0.5*b('B5')",
                             # "GVSP    = (-0.283*b('B3') - 0.66*b('B4') + 0.577*b('B6') + 0.388*b('B8'))/(0.433*b('B3') - 0.632*b('B4') + 0.586*b('B6') + 0.264*b('B8A'))",
                             # "MCARI   = ((b('B5')-b('B4'))-0.2*(b('B5')-b('B3')))*(b('B5')/b('B4'))",
                             # "TCARI   = 3*((b('B5')-b('B4'))-0.2*(b('B5')-b('B3'))*(b('B5')/b('B4')))",
                               "EVI     = 2.5*((b('B8')-b('B4'))/(b('B8')+6*b('B4')-7.5*b('B3')+1))",
                               "EVI2    = 2.5*((b('B8')-b('B4'))/(b('B8')+2.4*b('B4')+1))",
                               "RDVI    = (b('B8')-b('B4'))/((b('B8')+b('B4'))**0.5)",
                               "MSR     = ((b('B8')/b('B4'))-1)/((b('B8')/b('B4'))**0.5+1)",
                             # "MSAVI   = 0.5*(2*b('B7')+1-((2*b('B7')+1)**2-8*(b('B7')-b('B4')))**0.5)",
                               "MSAVI2  = 0.5*(2*b('B8')+1-((2*b('B8')+1)**2-8*(b('B8')-b('B4')))**0.5)",
                             # "MCARI2  = (1.5*(2.5*(b('B7')-b('B4'))-1.3*(b('B7')-b('B3'))))/((((2*b('B7')+1)**2)-(6*b('B7')-5*(b('B4')**0.5))-0.5)**0.5)",
                             # "MTVI2   = (1.5*(1.2*(b('B7')-b('B3'))-2.5*(b('B4')-b('B3'))))/(((2*b('B7')+1)**2-(6*b('B7')-5*b('B4'))-0.5)**0.5)",
                             # "MSR2    = ((b('B7')/b('B4'))-1)/(((b('B7')/b('B4'))+1)**0.5)",
                               "NLI     = ((b('B8')**2)-b('B4'))/((b('B8')**2)+b('B4'))"])

In [3]:
# inputImage = ee.Image('users/ccrs1fy2020/Test4')
# inputImage_bands = ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'B13', 'LAI'])

input_VI_definition = ee.List(["GI      = b('B3')/b('B4')",
                               "RVI3    = b('B4')/b('B6')",
                               "SR3     = b('B5')/b('B4')",
                               "GM1     = b('B6')/b('B3')",
                               "GM2     = b('B6')/b('B5')",
                               "SR2     = b('B7')/b('B3')",
                               "PSSR    = b('B7')/b('B4')",
                               "SGI     = b('B8A')/b('B4')",
                               "MSI     = b('B11')/b('B7')",
                               "II      = b('B11')/b('B12')",
                               "GVI     = (b('B8A')/b('B3'))-1",
                               "PSRI    = (b('B4')-b('B3'))/b('B6')",
                               "NDVI3   = ((b('B8A')-b('B4'))/(b('B8A')))+b('B4')",
                               "SR5     = 1/b('B5')",
                               "SR6     = b('B4')/(b('B3')*b('B5'))",
                               "SR7     = b('B8A')/(b('B3')*b('B5'))",
                               "IPVI    = b('B7')/(b('B7')+b('B4'))",
                               "ARI     = (1/b('B3'))-(1/b('B5'))",
                               "ARI2    = b('B7')*((1/b('B3'))-(1/b('B5')))",
                               "NDVI    = (b('B8A')-b('B4'))/(b('B8A')+b('B4'))",
                               "GNDVI   = (b('B8A')-b('B3'))/(b('B8A')+b('B3'))",
                               "NDWI    = (b('B8A')-b('B11'))/(b('B8A')+b('B11'))",
                               "NDREVI  = (b('B8A')-b('B5'))/(b('B8A')+b('B5'))",
                               "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",
                               "NDI1    = (b('B7')-b('B5'))/(b('B7')-b('B4'))",
                               "NDI2    = (b('B8A')-b('B5'))/(b('B8A')-b('B4'))",
                               "RENDVI  = (b('B6')-b('B5'))/(b('B6')+b('B5'))",
                               "OSAVI   = (1.16*(b('B7')-b('B4')))/(b('B7')+b('B4')+0.61)",
                               "NMDI    = (b('B8A')-(b('B11')-b('B12')))/(b('B8A')+(b('B11')-b('B12')))",
                               "HI      = ((b('B3')-b('B5'))/(b('B3')+b('B5')))-0.5*b('B5')",
                               "GVSP    = (-0.283*b('B3') - 0.66*b('B4') + 0.577*b('B6') + 0.388*b('B8A'))/(0.433*b('B3') - 0.632*b('B4') + 0.586*b('B6') + 0.264*b('B8A'))",
                               "MCARI   = ((b('B5')-b('B4'))-0.2*(b('B5')-b('B3')))*(b('B5')/b('B4'))",
                               "TCARI   = 3*((b('B5')-b('B4'))-0.2*(b('B5')-b('B3'))*(b('B5')/b('B4')))",
                               "EVI     = 2.5*((b('B8A')-b('B4'))/(b('B8A')+6*b('B4')-7.5*b('B3')+1))",
                               "EVI2    = 2.5*((b('B8A')-b('B4'))/(b('B8A')+2.4*b('B4')+1))",
                               "RDVI    = (b('B8A')-b('B4'))/((b('B8A')+b('B4'))**0.5)",
                               "MSR     = ((b('B8A')/b('B4'))-1)/((b('B8A')/b('B4'))**0.5+1)",
                               "MSAVI   = 0.5*(2*b('B7')+1-((2*b('B7')+1)**2-8*(b('B7')-b('B4')))**0.5)",
                               "MSAVI2  = 0.5*(2*b('B8A')+1-((2*b('B8A')+1)**2-8*(b('B8A')-b('B4')))**0.5)",
                               "MCARI2  = (1.5*(2.5*(b('B7')-b('B4'))-1.3*(b('B7')-b('B3'))))/((((2*b('B7')+1)**2)-(6*b('B7')-5*(b('B4')**0.5))-0.5)**0.5)",
                               "MTVI2   = (1.5*(1.2*(b('B7')-b('B3'))-2.5*(b('B4')-b('B3'))))/(((2*b('B7')+1)**2-(6*b('B7')-5*b('B4'))-0.5)**0.5)",
                               "MSR2    = ((b('B7')/b('B4'))-1)/(((b('B7')/b('B4'))+1)**0.5)",
                               "NLI     = ((b('B8A')**2)-b('B4'))/((b('B8A')**2)+b('B4'))"])

In [5]:
def format_image(image, image_bands, response_band, VI_definition):
    image = ee.Image(image)
    image_bands = ee.List(image_bands)
    response_band = ee.String(response_band)
    VI_definition = ee.List(VI_definition)
    
    # image_bands specifices a list of the names of the bands used in defining the expressions for VIs in VI_definition
    image = image.rename(image_bands).toDouble()
    
    # Generate an imageCollection from a list of expressions defining a set of Vegetation Indices using the bands available in the image
    VIimageCollection = ee.ImageCollection(VI_definition.map(lambda expr: image.expression(ee.String(expr))))
    VIimage = VIimageCollection.toBands().regexpRename('[0-9]+_', '')
    
    # Reordering the bands in the image so the response band is the last band in the image
    feature_bands = image_bands.remove(response_band)
    
    return image.select(feature_bands).addBands(VIimage).addBands(image.select(response_band))

In [6]:
inputImage = format_image(inputImage, inputImage_bands, defaultBand, input_VI_definition)

In [7]:
def get_num_pixels(image):
    # Retrieve the dimensions of the image from the metadata of the first band
    image_dimensions = ee.List(image.getInfo()['bands'][28]['dimensions'])
    image_height = image_dimensions.getNumber(0)
    image_width = image_dimensions.getNumber(1)
    image_pixels = image_height.multiply(image_width)
    
    return image_pixels

In [8]:
num_input_pixels = get_num_pixels(inputImage)

In [9]:
def scale_image(image, response_band):
    image = ee.Image(image)
    response_band = ee.String(response_band)
    image_pixels = ee.Number(get_num_pixels(image))
    
    # Setting up lists containing the input/feature bands in the image
    bandList = image.bandNames()
    featureList = bandList.remove(response_band)
    num_bands = bandList.length()
    num_features = featureList.length()
    
    # We will be using the reduceRegion() function on images from Earth Engine, 
    # which will process up to a specified number of pixels from the image to generate the outputs of the reducer
    max_pixels = image_pixels.min(10000000)
    # best_effort = ee.Algorithms.If(image_pixels.gt(max_pixels), True, False)
    
    # Set default projection and scale using the response band
    defaultScale = image.select(defaultBand).projection().nominalScale()
    defaultCrs = image.select(defaultBand).projection().crs()
    image = image.setDefaultProjection(crs=defaultCrs, scale=defaultScale)
    
    # Initial centering all of the input bands, added VIs, and response with the input image
    meanImage = image.subtract(image.reduceRegion(reducer=ee.Reducer.mean(), scale=defaultScale, bestEffort=True, maxPixels=max_pixels).toImage(bandList))
    
    # Separating the image into features (X) and response (y) for processing with LARs
    X = meanImage.select(featureList)
    y = meanImage.select(response_band)
    
    # Standardizing the input features
    X = X.divide(X.reduceRegion(reducer=ee.Reducer.stdDev(), bestEffort=True, maxPixels=max_pixels).toImage(featureList))
    
    return X.addBands(y)

In [10]:
scaledImage = scale_image(inputImage, defaultBand)

In [117]:
'''
def image2collection(image, num_samples):
    image = ee.Image(image)
    num_samples = ee.Number(num_samples)
    image_pixels = ee.Number(get_num_pixels(image))
    n = num_samples.min(image_pixels)
    
    # Extracting the image into arrays
    inputsCollection = image.sample(None, None, None, None, n, 0, True, 1, True)
    
    return inputsCollection
'''

'\ndef image2collection(image, num_samples):\n    image = ee.Image(image)\n    num_samples = ee.Number(num_samples)\n    image_pixels = ee.Number(get_num_pixels(image))\n    n = num_samples.min(image_pixels)\n    \n    # Extracting the image into arrays\n    inputsCollection = image.sample(None, None, None, None, n, 0, True, 1, True)\n    \n    return inputsCollection\n'

In [118]:
# inputsCollection = image2collection(scaledImage, 10000)

In [ ]:
# Export.table.toDrive({
#     collection: inputsCollection,
#     description: 'untrimmed_image_samples',
#     fileFormat: 'CSV',
#     selectors: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'B13',
#                 "GI", "RVI3", "SR3", "GM1", "GM2", "SR2", "PSSR", "SGI", "MSI", "II", "GVI", "PSRI", "NDVI3",
#                 "SR5", "SR6", "SR7", "IPVI", "ARI", "ARI2", "NDVI", "GNDVI", "NDWI", "NDREVI", "NDGI", "NDI1",
#                 "NDI2", "RENDVI", "OSAVI", "NMDI", "HI", "GVSP", "MCARI", "TCARI", "EVI", "EVI2","RDVI", "MSR",
#                 "MSAVI", "MSAVI2", "MCARI2", "MTVI2", "MSR2", "NLI", "LAI"]
#  })

In [11]:
def EE_LARS(image, response_band, num_nonzero_coefficients, num_samples):
    image = ee.Image(image)
    response_band = ee.String(response_band)
    num_nonzero_coefficients = ee.Number(num_nonzero_coefficients)
    num_samples = ee.Number(num_samples)
    bandList = image.bandNames()
    featureList = bandList.remove(response_band)
    image_pixels = get_num_pixels(image)
    inputCollection = image.sample(None, None, None, None, num_samples.min(image_pixels), 0, True, 1, True)
    
    n = inputCollection.size()
    m = featureList.length()
    
    inputs = ee.Dictionary.fromLists(bandList, bandList.map(lambda feature: inputCollection.aggregate_array(feature)))
    
    # Re-centering the features and response as reduceRegion() is not as precise as centering with arrays 
    input_means = ee.Dictionary.fromLists(bandList, bandList.map(lambda feature: inputCollection.aggregate_mean(feature)))
    
    def center_inputs(key, value):
        key_mean = input_means.getNumber(key)
        return ee.List(value).map(lambda sample: ee.Number(sample).subtract(key_mean))
    
    inputs = inputs.map(center_inputs)
    y = inputs.toArray([response_band]).reshape([-1,1])

    # Re-normalizing the input features as reduceRegion() is not as precise as normalizing with arrays
    inputs = inputs.select(featureList)
    input_norms = inputs.map(lambda key, value: ee.Number(ee.List(value).map(lambda sample: ee.Number(sample).pow(2)).reduce(ee.Reducer.sum())).pow(0.5))
    
    def norm_inputs(key, value):
        key_norm = input_norms.getNumber(key)
        return ee.List(value).map(lambda sample: ee.Number(sample).divide(key_norm))
    
    inputs = inputs.map(norm_inputs)
    X = inputs.toArray(featureList).transpose()
    
    # Find the first most correlated variable to pass into the main loop
    initial_prediction = ee.Array(ee.List.repeat([0], n))
    c = X.transpose().matrixMultiply(y.subtract(initial_prediction))
    c_abs = c.abs()
    C_maxLoc = c_abs.project([0]).argmax()
    add_feature = C_maxLoc.getNumber(0)
    A = ee.List([add_feature])
    
    initial_inputs = ee.Dictionary({'prediction': initial_prediction, 'A': A})
    # print(A.getInfo())
    # print(initial_inputs.getInfo())
    
    # Main loop
    def LARs_regression(iteration, inputs):
        inputs = ee.Dictionary(inputs)
        A = ee.List(inputs.get('A'))
        A_list = ee.Array(ee.List.sequence(0, m.subtract(1)).map(lambda index: A.contains(index))\
                          .replaceAll(False, 0).replaceAll(True ,1)).reshape([-1,1])

        prediction = inputs.getArray('prediction')
        c = X.transpose().matrixMultiply(y.subtract(prediction))
        c_abs = c.abs()
        C_max = c_abs.get(c_abs.argmax())

        s_A = c.divide(c_abs).mask(A_list)
        X_A = X.mask(A_list.transpose())
        G_Ai = X_A.transpose().matrixMultiply(X_A).matrixInverse()
        G1 = G_Ai.matrixMultiply(s_A)
        A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
        w_A = G1.multiply(A_A)
        u_A = X_A.matrixMultiply(w_A)
        a = X.transpose().matrixMultiply(u_A)

        a = a.project([0])
        c = c.project([0])
        
        def compute_gammaArray(index_j):
            minus_j = C_max.subtract(c.get([index_j])).divide(A_A.subtract(a.get([index_j])))
            plus_j = C_max.add(c.get([index_j])).divide(A_A.add(a.get([index_j])))
            return ee.List([minus_j, plus_j]).filter(ee.Filter.gte('item', 0)).reduce(ee.Reducer.min())
    
        A_c = ee.List.sequence(0, m.subtract(1)).removeAll(A)
        gammaArray = A_c.map(compute_gammaArray)
        gamma = gammaArray.reduce(ee.Reducer.min())
        min_location = gammaArray.indexOf(gamma)
        add_feature = A_c.getNumber(min_location)
        A = A.add(add_feature)
        prediction = prediction.add(u_A.multiply(gamma))

        return ee.Dictionary({'prediction': prediction, 'A': A})

    def LARs_final_iteration(iteration, inputs):
        inputs = ee.Dictionary(inputs)
        A = ee.List(inputs.get('A'))

        prediction = inputs.getArray('prediction')
        c = X.transpose().matrixMultiply(y.subtract(prediction))
        c_abs = c.abs()
        C_max = c_abs.get(c_abs.argmax())

        s_A = c.divide(c_abs)
        G_Ai = X.transpose().matrixMultiply(X).matrixInverse()
        G1 = G_Ai.matrixMultiply(s_A)
        A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
        w_A = G1.multiply(A_A)
        u_A = X.matrixMultiply(w_A)

        gamma = C_max.divide(A_A)
        prediction = prediction.add(u_A.multiply(gamma))

        return ee.Dictionary({'prediction': prediction, 'A': A})
    
    iterations = ee.List.sequence(1, m.subtract(1).min(num_nonzero_coefficients))
    penultimate_outputs = iterations.iterate(LARs_regression, initial_inputs)
    final_outputs = ee.Dictionary(ee.Algorithms.If(num_nonzero_coefficients.gte(m), 
                                                      LARs_final_iteration(m, penultimate_outputs),
                                                      penultimate_outputs))
    
    final_prediction = final_outputs.getArray('prediction')
    A = ee.List(final_outputs.get('A'))
    feature_path = A.slice(0, num_nonzero_coefficients).map(lambda index: featureList.getString(index))
    
    coefficients = X.matrixSolve(final_prediction)
    
    def set_zero(num):
        num = ee.Number(num)
        return ee.Algorithms.If(num.abs().lt(0.001), 0, num)
    
    coefficients = coefficients.project([0]).toList().map(lambda num: set_zero(num))
    # print('Coefficients', ee.Dictionary.fromLists(featureList, coefficients))
    
    return feature_path

In [12]:
select_features = ee.List(EE_LARS(scaledImage, defaultBand, 5, 10000)).sort()

In [13]:
# The following function trims input data according to an algorithm in which the response band is partitioned into n equally sized
# partitions, and in each of the n partitions, for the features selected by LARs, they are each trimmed individually down to only the
# 5-95 percentile of the data. We are not doing any preprocessing with the data, so the raw data is exported from Earth Engine
# The function takes an image, a list of strings with the selected feature bands in the image, the string that is the name of the response
# band in this image, the number of samples/pixels the user wants to take from the image, and the number of parititions to trim within
def trim_data(image, selected_features, response_band, num_samples, num_partitions):
    image = ee.Image(image)
    selected_features = ee.List(selected_features)
    response_band = ee.String(response_band)
    num_samples = ee.Number(num_samples)
    num_partitions = ee.Number(num_partitions)
    
    # Generate the list of percentile bounds for the requested number of partitions, and the names of the value bounds for the
    # dictionary that will be generated from the percentile reducer used later on
    percentiles = ee.List.sequence(0, 100, ee.Number(100).divide(num_partitions))
    percentile_names = percentiles.map(lambda num: ee.Number(num).round().toInt().format("p%s"))

    # Randomly sample the pixels in the input image into a feature collection containing only the selected features and the response
    image_pixels = ee.Number(get_num_pixels(image))
    inputsCollection = image.select(selected_features.add(response_band)).sample(numPixels=num_samples.min(image_pixels))

    # Find the values at the percentile bounds using the percentile reducer over the feature collection
    response_percentiles = inputsCollection.reduceColumns(ee.Reducer.percentile(percentiles=percentiles, outputNames=percentile_names, maxRaw=inputsCollection.size()), [response_band])
    
    # Create a list of percentile bounds for each partition
    response_partitions = response_percentiles.values(percentile_names.remove('p100')).zip(response_percentiles.values(percentile_names.remove('p0')))
    
    def partition_data(partition_range):
        partition_range = ee.List(partition_range)
        return inputsCollection.filter(ee.Filter.rangeContains(response_band, partition_range.getNumber(0), partition_range.getNumber(1)))
    
    partitioned_data = response_partitions.map(partition_data)

    # The following function now trims the data in each partition individually for each feature to its 5-95 percentile only
    def trim_partitions(partition):
        partition = ee.FeatureCollection(partition)
        feature_trimming_bounds = selected_features.map(lambda feature: ee.List([feature]).cat(partition.reduceColumns(ee.Reducer.percentile([5, 95]), [feature]).values(['p5','p95'])))
        def trimmer(current_feature, collection):
            current_feature = ee.List(current_feature)
            collection = ee.FeatureCollection(collection)
            return collection.filter(ee.Filter.rangeContains(current_feature.getString(0), current_feature.getNumber(1), current_feature.getNumber(2)))
        return feature_trimming_bounds.iterate(trimmer, partition)
    
    # Retrieve the trimmed data partitions and flatten the paritions into a single trimmed feature collection
    trimmed_partitions = partitioned_data.map(trim_partitions)
    trimmed_data = ee.FeatureCollection(trimmed_partitions).flatten()
    
    return trimmed_data

In [14]:
trimmedCollection = trim_data(image=inputImage.updateMask(inputImage.select(defaultBand).gt(0)), \
                    selected_features=select_features, response_band=defaultBand, num_samples=5000, num_partitions=10)

In [15]:
exportData = ee.batch.Export.table.toDrive(collection=trimmedCollection, description='image_samples', fileFormat='CSV')
exportData.start()

In [16]:
# Essentially a wait loop to see if the data has finished exporting by checking with the server-side
prev_task_status = ee.data.getTaskStatus(exportData.id)[0]["state"]
print(prev_task_status)
while exportData.active():
    task_status = ee.data.getTaskStatus(exportData.id)[0]["state"]
    if(task_status != prev_task_status):
        print(task_status)
    prev_task_status = task_status
    time.sleep(5)
print(ee.data.getTaskStatus(exportData.id)[0]["state"])

READY
RUNNING
COMPLETED


In [17]:
nnet = ee.FeatureCollection("users/hemitshah/nnet5")
nnet_inputs = nnet.filter(ee.Filter.eq("layer_num", 0)).first()
num_inputs = nnet_inputs.getNumber("num_nodes")

selected_features = nnet_inputs.getString("activation").split(",")

nnet = nnet.filterBounds(ee.Geometry.Point([0,0]))
layer_list = nnet.sort("layer_num").toList(nnet.size())

In [18]:
def parse_layer(feature):
    feature = ee.Feature(feature)
    prev_layer_size = feature.getNumber("prev_layer_size")
    num_nodes = feature.getNumber("num_nodes")
    node_size = prev_layer_size.subtract(1)
    activation = feature.getString("activation")
    
    node_collection = ee.ImageCollection(ee.List.sequence(1, num_nodes)\
                        .map(lambda node: ee.ImageCollection(ee.List.sequence(ee.Number(node).toInt(), ee.Number(node)\
                                    .toInt().add(node_size.multiply(num_nodes)), num_nodes)\
                        .map(lambda index: ee.Image(feature.getNumber(ee.Number(index).toInt())))).toBands()\
                        .set({"bias": feature.getNumber(ee.Number(node).toInt().add(prev_layer_size.multiply(num_nodes)))})))
    
    return ee.List([node_collection, activation])

In [19]:
neural_net = layer_list.map(parse_layer)
'''
print(ee.List(neural_net.get(0)).get(0))
print(ee.List(neural_net.get(0)).get(1))
print(ee.List(neural_net.get(1)).get(0))
print(ee.List(neural_net.get(1)).get(1))
print(ee.List(neural_net.get(2)).get(0))
print(ee.List(neural_net.get(2)).get(1))
print(ee.List(neural_net.get(3)).get(0))
print(ee.List(neural_net.get(3)).get(1))
print(ee.List(neural_net.get(4)).get(0))
print(ee.List(neural_net.get(4)).get(1))
'''

'\nprint(ee.List(neural_net.get(0)).get(0))\nprint(ee.List(neural_net.get(0)).get(1))\nprint(ee.List(neural_net.get(1)).get(0))\nprint(ee.List(neural_net.get(1)).get(1))\nprint(ee.List(neural_net.get(2)).get(0))\nprint(ee.List(neural_net.get(2)).get(1))\nprint(ee.List(neural_net.get(3)).get(0))\nprint(ee.List(neural_net.get(3)).get(1))\nprint(ee.List(neural_net.get(4)).get(0))\nprint(ee.List(neural_net.get(4)).get(1))\n'

In [20]:
def linear(x):
    return ee.Image(x)

def elu(x):
    x = ee.Image(x)
    return ee.ImageCollection([x.mask(x.gte(0)), x.mask(x.lt(0)).exp().subtract(1)]).mosaic()

def softplus(x):
    x = ee.Image(x)
    return x.exp().add(1).log()

def softsign(x):
    x = ee.Image(x)
    return x.divide(x.abs().add(1))

def relu(x):
    x = ee.Image(x)
    return x.max(0.0)

def tanh(x):
    x = ee.Image(x)
    return x.multiply(2).exp().subtract(1).divide(x.multiply(2).exp().add(1))

def sigmoid(x):
    return x.exp().pow(-1).add(1).pow(-1)

In [21]:
def apply_nnet(layer, net_input):
    layer = ee.List(layer)
    net_input = ee.Image(net_input)
    
    layer_nodes = ee.ImageCollection(layer.get(0))
    activation = layer.getString(1)
    
    node_outputs = layer_nodes.map(lambda node: ee.Algorithms.If(activation.compareTo("linear"), \
                        softsign(net_input.multiply(node).reduce(ee.Reducer.sum()).add(node.getNumber("bias"))), \
                        net_input.multiply(ee.Image(node)).reduce(ee.Reducer.sum()).add(ee.Image(node).getNumber("bias")))).toBands()

    return node_outputs

In [22]:
validation_data = inputImage.select("LAI")
nnet_inputs = scaledImage.select(selected_features)
# layer1out = apply_nnet(neural_net.get(0), nnet_inputs)
# layer2out = apply_nnet(neural_net.get(1), layer1out)
# layer3out = apply_nnet(neural_net.get(2), layer2out)
# layer4out = apply_nnet(neural_net.get(3), layer3out)
# layer5out = apply_nnet(neural_net.get(4), layer4out)
# Map.addLayer(layer1out)
# Map.addLayer(layer2out)
# Map.addLayer(layer3out)
# Map.addLayer(layer4out)
# Map.addLayer(layer5out)

In [23]:
prediction_data = ee.Image(neural_net.iterate(apply_nnet, nnet_inputs)).rename("NNET")
inputsCollection = ee.FeatureCollection("users/hemitshah/image_data_samples").select(selected_features.add("LAI"))

In [24]:
ee_regressor = ee.Classifier.smileRandomForest(numberOfTrees=100,
                                               variablesPerSplit=0,
                                               minLeafPopulation=3,
                                               bagFraction=0.1, seed=0).setOutputMode("REGRESSION")\
                .train(features=inputsCollection, classProperty="LAI", inputProperties=selected_features)

In [25]:
ee_prediction = nnet_inputs.addBands(validation_data).classify(ee_regressor, "RANDOM_FOREST")
# Map.addLayer(validation_data)
# Map.addLayer(prediction_data)
# Map.addLayer(ee_prediction)

In [26]:
nnet_rmse = prediction_data.subtract(validation_data).pow(2).reduceRegion(ee.Reducer.mean(), None, None, None, None, True, 10000000, 1).values().getNumber(0).pow(0.5)
rf_rmse = ee_prediction.subtract(validation_data).pow(2).reduceRegion(ee.Reducer.mean(), None, None, None, None, True, 10000000, 1).values().getNumber(0).pow(0.5)
# print("NEURAL NET RMSE: ", nnet_rmse)
# print("RANDOM FOREST RMSE: ", rf_rmse)